## make the cells wider

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

## Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import csv
import requests
import sqlite3

### Establishing connection between python and sqlite data base

In [2]:
connection = sqlite3.connect("books.db")

### Functions used in main function

In [3]:
def initializing_page(url): ## getting the body's html code
    req = requests.get(url).text
    soup = BeautifulSoup(req,'html.parser')
    body = soup.body
    return body

In [4]:
def Grap_categories(url): # Grap Each category name assigned with its link
    body = initializing_page(url)
    ul = body.find('ul',attrs = {'class': 'nav nav-list'})
    li = ul.find('li').find('ul').find_all('li')
    categories = {i.text.strip():('https://books.toscrape.com/'+ i.find('a').attrs['href']) for i in li }
    return categories

In [5]:
def grap_page_books(url): # grap books detail for each page
    numbers = {'One':1,'Two':2,'Three':3, 'Four':4,'Five':5}
    body = initializing_page(url)
    all_books = body.find_all('li',attrs = {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    books_names =[i.find('h3').find('a').attrs['title'].replace('â\x80\x99','\'') for i in all_books]
    rates = [numbers[i.find('p').attrs['class'][-1]] for i in all_books]
    price = [float(i.find('p',attrs={'class': 'price_color'}).text.split('£')[-1]) for i in all_books]
    return list(zip(books_names,price,rates))

In [6]:
def no_books_inpage(url): # return the number of book in the given url
    body = initializing_page(url)
    result = int(body.find('form',{'method': 'get','class':'form-horizontal'}).find('strong').text)
    return result

In [7]:
def initiate_categories(categories): # inserting the categories into the categories table in data base
    for i in categories:
        cursor = connection.execute(f"INSERT INTO categories (category_name) VALUES ('{i}')")
    connection.commit()

In [8]:
# main function that graps the books details and categories names  
# inserting it into the books and categories tabels in data base
def main(): 
    urls = list(Grap_categories('https://books.toscrape.com/').values())
    categories = list(Grap_categories('https://books.toscrape.com/').keys())
    initiate_categories(categories)
    insert_query = "INSERT INTO books (book_name,book_price,book_rate,category_id) VALUES (?,?,?,?)"
    for i in range(len(urls)):
        li1 = grap_page_books(urls[i])
        for y in range(len(li1)):
            li2 = list(li1[y])
            book_name = "\"{}\"".format(li2[0].replace('\"','\''))
            category_id = connection.execute(f"select category_id from categories where category_name = '{categories[i]}'").fetchall()[0][0]
            cursor = connection.execute(insert_query, [book_name, li2[1],li2[2],category_id])
            connection.commit()
        if no_books_inpage(urls[i]) > 20 : # checks if the category holds more than 20 books so it turns to the left pages
            result = (no_books_inpage(urls[i]) // 20) + 2 if no_books_inpage(urls[i]) % 20 != 0 else (no_books_inpage(urls[i]) // 20) + 1
            for j in range(2,result):
                li3 = grap_page_books(urls[i].replace('index',f'page-{j}'))
                for z in range(len(li3)):
                        li4 = list(li3[z])
                        cursor = connection.execute(insert_query, [book_name, li4[1],li4[2],category_id])
                        connection.commit()
    connection.close()
    print('mission succed!')

In [9]:
main()

mission succed!


## By Anas Awad